# Environment Setting Up

In [1]:
import os
from dotenv import load_dotenv

# Loading environment variables from .env
load_dotenv()

# Changing directory to main directory for easy data access
working_directory = os.getenv("WORKING_DIRECTORY")
os.chdir(working_directory)

# Checking the change
%pwd

'/workspaces/TumorTracer'

In [2]:
from pathlib import Path

# Checking the change
print("Git folder exists:", Path(".git").exists())

Git folder exists: True


# 2. Base Model

In [3]:
from dataclasses import dataclass
from pathlib import Path
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

@dataclass(frozen=True)
class BaseModelConfig:
    """
    Immutable configuration class to hold all necessary paths 
    and parameters required for the base model stage.
    """
    root_dir: Path                              # Directory where models will be stored
    model_path: Path                            # Path to the downloaded base model
    updated_model_path: Path                    # Path to the updated model with custom layers
    params_image_size: tuple[int, int, int]     # Input image size, e.g., [224, 224, 3]
    params_include_top: bool                    # Whether to include fully connected layers
    params_classes: int                         # Number of output classes
    params_weights: str                         # Pre-trained weights source
    params_learning_rate: float                 # Learning rate for training

In [4]:
from cnnClassifier.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class ConfigurationManager:
    def __init__(self, config_file_path=CONFIG_FILE_PATH, params_file_path=PARAMS_FILE_PATH) -> None:
        """
        Reads configuration files (config.yaml and params.yaml), 
        ensures necessary directories exist, and prepares structured config objects.

        Args:
        - config_file_path (str): Path to the config.yaml file.
        - params_file_path (str): Path to the params.yaml file.
        """
        # Validate and load config.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Config file not found at: {config_file_path}")
            raise FileNotFoundError(f"Config file not found at: {config_file_path}")
        self.config = read_yaml(config_file_path)

        # Validate and load params.yaml
        if not Path(config_file_path).exists():
            logger.error(f"Params file not found at: {params_file_path}")
            raise FileNotFoundError(f"Params file not found at: {params_file_path}")
        self.params = read_yaml(params_file_path)

        logger.info(f"Loading configuration from {config_file_path} and parameters from {params_file_path}")

        # Create the root artifacts directory (if not already present)
        create_directories([self.config.artifacts_root])

    def get_base_model_config(self) -> BaseModelConfig:
        """
        Prepares and returns the BaseModelConfig object.

        Returns:
        - BaseModelConfig: Structured config for downloading and updating base model.
        """
        config = self.config.base_model
        params = self.params.base_model

        # Ensure the data_ingestion directory exists
        create_directories([config.root_dir])

        # Build and return a structured configuration object for base model construction
        training_config = BaseModelConfig(
            root_dir=Path(config.root_dir),
            model_path=Path(config.model_path),
            updated_model_path=Path(config.updated_model_path),
            params_image_size=tuple(params.IMAGE_SIZE),             # Convert list to tuple for immutability
            params_include_top=params.INCLUDE_TOP,
            params_classes=params.CLASSES,
            params_weights=params.WEIGHTS,
            params_learning_rate=params.LEARNING_RATE,
        )
        
        logger.info(f"BaseModelConfig created with: {training_config}")

        return training_config

In [11]:
import tensorflow as tf
from cnnClassifier.utils.common import create_directories
from cnnClassifier import get_logger

# Initializing the logger
logger = get_logger()

class BaseModelConstruction:
    """
    Handles the retrieval and customization of the base model (VGG16) for transfer learning.

    Responsibilities:
    - Downloads a pretrained VGG16 model
    - Appends custom dense layers for classification (if called)
    - Saves both base and updated models

    Attributes:
    - config (BaseModelConfig): Configuration object with model paths and hyperparameters

    Public Methods:
    - get_model(): Downloads and saves the base VGG16 model
    - updated_model(): Adds custom dense layers and saves the modified model

    Internal Methods:
    - _prepare_model(): Modifies the base model for the current classification task
    - _save_model(): Saves the model to the specified path
    """
    def __init__(self, config: BaseModelConfig) -> None:
        self.config = config
        self.model = None
        self.enhanced_model = None


    def get_model(self) -> None:
        """
        Downloads the pretrained the pretrained VGG16 model and saves it.
        """
        try:
            logger.info(f"Downloading base VGG16 model...")
            self.model = tf.keras.applications.vgg16.VGG16(
                input_shape=self.config.params_image_size,
                weights=self.config.params_weights,
                include_top=self.config.params_include_top,
            )

            logger.info(f"Successfully downloaded VGG16 base model.")
            self._save_model(save_path=self.config.model_path, model=self.model)
        
        except Exception as exception_error:
            logger.error(f"Unexpected error while downloading the base model: {exception_error}")
            raise 


    def updated_model(self) -> None:
        """
        Updates the downloaded model with custom dense layers and saves it.
        """
        if self.model is None:
            logger.error("Base model not found. Run get_model() before calling updated_model().")
            raise ValueError("Base model not found. Run get_model() before calling updated_model().")

        try: 
            logger.info("Preparing the enhanced model with custom dense layers...")

            self.enchanced_model = self._prepare_model(
                model=self.model,
                classes=self.config.params_classes,
                freeze_all=True,
                freeze_till=None,
                learning_rate=self.config.params_learning_rate,
            )

            logger.info("Successfully created the enhanced model with custom dense layers.")
            self._save_model(save_path=self.config.updated_model_path, model=self.enchanced_model)

        except Exception as exception_error:
            logger.error(f"Unexpected error while creating the enchanced model: {exception_error}")
            raise 

    @staticmethod
    def _prepare_model(model: tf.keras.Model, classes: int, freeze_all: bool, freeze_till: int, learning_rate: float) -> tf.keras.Model:
        """
        Customizes the base model by freezing layers and adding classification head.

        Parameters:
        - model (tf.keras.Model): Pretrained base model
        - classes (int): Number of output classes
        - freeze_all (bool): Whether to freeze all layers
        - freeze_till (int): Number of layers from the end to remain trainable
        - learning_rate (float): Learning rate for optimizer

        Returns:
        - tf.keras.Model: Fully compiled transfer learning model
        """
        # Make all layers trainable first (so we can selectively freeze them)
        model.trainable = True

        # If freeze_all is True, freeze the entire base model
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        # Optionally freeze up to a certain layer, allowing fine-tuning of last few
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        # Converts feature maps to a 1D vector
        flatten_model = tf.keras.layers.Flatten()(model.output)
        
        # Adds output neurons for each class.
        prediction = tf.keras.layers.Dense(units=classes, activation="softmax")(flatten_model)

        # Wraps the base model and the new classification head into one Model
        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()
        return full_model
    
    
    def _save_model(self, save_path: Path, model: tf.keras.Model) -> None:
        """
        Saves a given model to the specified path.
        """
        try:
            create_directories([save_path.parent])
            model.save(save_path)
            logger.info(f"Model saved at: {save_path}")
        
        except Exception as exception_error:
            logger.error(f"Unexpected error while saving the model at {save_path}: {exception_error}")
            raise 

In [12]:
try:
    config_manager = ConfigurationManager()
    base_model_config = config_manager.get_base_model_config()

    base_model_constructor = BaseModelConstruction(config=base_model_config)
    base_model_constructor.get_model()
    base_model_constructor.updated_model()

except Exception as exception:
    logger.exception(f"Unexpected error during data ingestion pipeline: {exception}")
    raise exception

[2025-07-02 12:40:14,267: INFO: common: YAML file: config/config.yaml loaded successfully]
[2025-07-02 12:40:14,277: INFO: common: YAML file: params.yaml loaded successfully]
[2025-07-02 12:40:14,279: INFO: 29643782: Loading configuration from config/config.yaml and parameters from params.yaml]
[2025-07-02 12:40:14,280: INFO: common: Directory: artifacts created successfully.]
[2025-07-02 12:40:14,281: INFO: common: Directory: artifacts/base_model created successfully.]
[2025-07-02 12:40:14,282: INFO: 29643782: BaseModelConfig created with: BaseModelConfig(root_dir=PosixPath('artifacts/base_model'), model_path=PosixPath('artifacts/base_model/base_model.h5'), updated_model_path=PosixPath('artifacts/base_model/updated_base_model.h5'), params_image_size=(224, 224, 3), params_include_top=False, params_classes=4, params_weights='imagenet', params_learning_rate=0.01)]
[2025-07-02 12:40:14,283: INFO: 4255369684: Downloading base VGG16 model...]
[2025-07-02 12:40:14,583: INFO: 4255369684: Succ

[2025-07-02 12:40:14,676: INFO: 4255369684: Model saved at: artifacts/base_model/base_model.h5]


INFO:cnnClassifierLogger_running:Model saved at: artifacts/base_model/base_model.h5


[2025-07-02 12:40:14,681: INFO: 4255369684: Preparing the enhanced model with custom dense layers...]


INFO:cnnClassifierLogger_running:Preparing the enhanced model with custom dense layers...


...

INFO:cnnClassifierLogger_running:Successfully created the enhanced model with custom dense layers.


[2025-07-02 12:40:14,728: INFO: common: Directory: artifacts/base_model created successfully.]


INFO:cnnClassifierLogger_test:Directory: artifacts/base_model created successfully.


[2025-07-02 12:40:14,853: INFO: 4255369684: Model saved at: artifacts/base_model/updated_base_model.h5]


INFO:cnnClassifierLogger_running:Model saved at: artifacts/base_model/updated_base_model.h5
